In [4]:
import os, sys, argparse, importlib, time, inspect, json
import numpy as np
import matplotlib.pyplot as plt
import os.path as osp
if hasattr(__builtins__,'__IPYTHON__'):
    print('Notebook')
    from tqdm.notebook import tqdm
else:
    print('Not notebook')
    from tqdm import tqdm
from tensorflow.keras import backend as K
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd

# gpu_devices = tf.config.list_physical_devices('GPU') 
# if len(gpu_devices) > 0:
#     print("GPU detected")
#     for i in range(len(gpu_devices)):
#         tf.config.experimental.set_memory_growth(gpu_devices[i], True)
# else:
#     print('No GPU detected')
import spektral
from sklearn.preprocessing import normalize
from spektral.data import DisjointLoader, BatchLoader, SingleLoader
from importlib import reload
os.chdir('../from_config')
import dev.data_load as dl
reload(dl)
graph_data=dl.graph_data

Notebook
Not notebook
Not notebook


In [6]:
os.chdir('../model_eval')

In [7]:
with tf.device('/cpu:0'): # if on the cpu
    model=tf.keras.models.load_model('../from_config/trained_models/IceCube/Sage_sage1nonorm_10_2aauycmh')
    model.compile()

In [6]:
batch_size=512
#just give the same database as you would normally run it on
dataset =graph_data(n_data=100000,skip=0, restart=1, transform=True,\
                    transform_path='db_files/muongun/transformers.pkl', db_path= 'db_files/muongun/rasmus_classification_muon_3neutrino_3mio.db')

Removed and ready to reload
Connecting to db-file
Loading Muons
ERROR! Session/line number was not unique in database. History logging moved to new session 1394
Reading files


C:\Users\chris\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\chris\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RobustScaler from version 0.19.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Splitting data to events


  0%|                                                                           | 136/100000 [00:00<01:13, 1349.52it/s]

     dom_x    dom_y    dom_z      time  charge_log10  SRTInIcePulses
0  0.11439 -0.46199 -0.14812  1.622453     -0.621035               0
1  0.13203  0.20298  0.12828 -0.060393     -1.921554               1
2  0.13203  0.20298  0.11126 -0.000741     -1.656923               1
3  0.38235  0.23890 -0.38653 -0.212301     -0.280277               0
4  0.50527  0.25788 -0.37884 -1.727306      0.162615               0
   energy_log10    zenith   azimuth
0      2.437721  0.661937  1.697274
1      2.584876  0.205041  4.291144
2      2.476674  0.661661  1.374715
3      2.409273  0.390813  2.876407
4      2.356129  0.889406  2.381056
Generating adjacency matrices


100%|████████████████████████████████████████████████████████████████████████| 100000/100000 [01:10<00:00, 1420.47it/s]


Saving dataset
Loading data to memory


In [33]:
df_event=dataset.df_event
train, val, test=dataset.index_lists
dataset_test=dataset[test]
loader = DisjointLoader(dataset_test, batch_size=batch_size, epochs=1)

In [29]:
@tf.function(input_signature = loader.tf_signature(), experimental_relax_shapes = True)
def test_step(inputs, targets):
    predictions = model(inputs, training = False)
    targets     = tf.cast(targets, tf.float32) 

    return predictions, targets

In [30]:
def predict(loader):
    prediction_list, target_list = [], []
    for batch in loader:
        inputs, targets = batch
        predictions, targets = test_step(inputs, targets)

        prediction_list.append(predictions.numpy())
        target_list.append(targets.numpy())
        y_reco  = tf.concat(prediction_list, axis = 0).numpy()
        y_true  = tf.concat(target_list, axis = 0)
        y_true  = tf.cast(y_true, tf.float32).numpy()
    return y_reco, y_true

In [31]:
reco, true=predict(loader)

In [35]:
reco_str=['energy_log10_pred', 'zenith_pred', 'azimuth_pred', 'zenith_sigma', 'azimuth_sigma']
recos=pd.DataFrame(reco)
recos.columns=reco_str
recos.head() ### change azi/zeni sigma to real sig

,energy_log10_pred,zenith_pred,azimuth_pred,zenith_sigma,azimuth_sigma
0,2.188606,0.736117,2.854449,3.116380,40.258015
1,2.392092,0.694575,2.097234,13.151760,123.957085
2,2.709592,0.167352,5.992949,2.947436,290.215179
3,2.680294,0.603433,4.233559,5.062509,32.112453
4,2.274279,1.069631,5.443553,3.917409,29.682205


In [42]:
recos['event_no']=np.array(df_event['event_no'][test])

In [ ]:
recos.head()
recos.to_csv('predictions_')